In [26]:
import numpy as np
import pandas as pd
import math
import csv

In [27]:
#Lê um arquivo de texto chamado "Lista-de-Palavras.txt" e armazena em um objeto do tipo DataFrame utilizando a biblioteca Pandas. 
#O parâmetro header=None indica que não há cabeçalho no arquivo e names=['Palavras'] indica que a única coluna deve ser chamada de "Palavras". 
#O parâmetro dtype=str especifica que o tipo de dado das colunas é string, encoding='utf-8' define a codificação do arquivo e 
#quoting=csv.QUOTE_NONE indica que não deve ser utilizada nenhuma citação para separar os campos. Por fim, na_filter=False desabilita a filtragem de valores nulos.
df = pd.read_csv('Lista-de-Palavras.txt', header=None, names=['Palavras'], dtype=str, encoding='utf-8', quoting=csv.QUOTE_NONE, na_filter=False)

#palavras_banco é uma lista que contém todas as palavras do arquivo "Lista-de-Palavras.txt". 
#palavras_banco_sh é uma lista que contém todas as palavras do arquivo "Lista-de-Palavras.txt" que não contêm o caractere '-'. Isso estava dando um problema.
palavras_banco = df['Palavras'].tolist()
palavras_banco_sh = [palavra for palavra in df['Palavras'] if '-' not in palavra]

#Lista de palavras incorretas que serão corrigidas.
palavras_incorretas = ['BERINJALA', 'ONOMATOPEYA', 'AXUL', 'PARALIZAR', 'ESPOJAR', 'IMPECILHO', 'UVAAZ', 'EXPECIME', 'PROESTANTISMO', 'DESECAR']

In [28]:
#nicialização de listas vazias que vão armazenar as correções de cada palavra incorreta.
correcoes_lev = []
correcoes_ang = []
correcoes_euc = []


In [29]:
#Função que calcula a distância de Levenshtein entre duas palavras.
def distancia_levenshtein(palavra1, palavra2):
    m, n = len(palavra1), len(palavra2)
    d = np.zeros((m+1, n+1), dtype=float) # altera o tipo de dado para float
    d[:, 0] = range(m+1)
    d[0, :] = range(n+1)
    for j in range(1, n+1):
        for i in range(1, m+1):
            if palavra1[i-1] == palavra2[j-1]:
                custo = 0
            else:
                custo = 1
            d[i, j] = min(d[i-1, j] + 1,          # deletar
                          d[i, j-1] + 1,          # inserir
                          d[i-1, j-1] + custo)    # substituir
    return d[m, n]

In [30]:
def distancia_euclidiana(palavra1, palavra2):
    # Ordenando o alfabeto pela tabela ASCII
    alphabet = sorted(list(set(palavra1 + palavra2)))
    d = np.zeros((len(alphabet),))
    for i, letter in enumerate(alphabet):
        d[i] = (palavra1.count(letter) - palavra2.count(letter))**2
    return np.sqrt(d.sum())

In [31]:
def distancia_angular(palavra1, palavra2):
    # Ordenando o alfabeto pela tabela ASCII
    alphabet = sorted(list(set(palavra1 + palavra2)))
    s_vec = np.array([palavra1.count(letter) for letter in alphabet])
    t_vec = np.array([palavra2.count(letter) for letter in alphabet])
    s_norm = s_vec / np.linalg.norm(s_vec)
    t_norm = t_vec / np.linalg.norm(t_vec)
    cos_theta = np.dot(s_norm, t_norm)
    return np.arccos(cos_theta)

In [32]:
#Função que retorna as cinco menores distâncias e suas respectivas palavras
def obter_cinco_menores(distancias, palavras):
    df = pd.DataFrame({"palavra": palavras, "distancia": distancias})
    df_ordenado = df.sort_values(by="distancia")
    cinco_menores = df_ordenado.head(5)
    return list(zip(cinco_menores["palavra"], cinco_menores["distancia"]))

In [33]:
def corrigir_palavra(palavra_incorreta):
    distancias_lev = [distancia_levenshtein(palavra_incorreta, palavra_banco) for palavra_banco in palavras_banco_sh]
    distancias_ang = [distancia_angular(palavra_incorreta, palavra_banco) for palavra_banco in palavras_banco_sh]
    distancias_euc = [distancia_euclidiana(palavra_incorreta, palavra_banco) for palavra_banco in palavras_banco_sh]
    correcoes_lev = obter_cinco_menores(distancias_lev, palavras_banco_sh)
    correcoes_ang = obter_cinco_menores(distancias_ang, palavras_banco_sh)
    correcoes_euc = obter_cinco_menores(distancias_euc, palavras_banco_sh)
    return (correcoes_lev, correcoes_ang, correcoes_euc)

In [34]:
correcoes = [corrigir_palavra(palavra_incorreta) for palavra_incorreta in palavras_incorretas]

# Extrai as informações necessárias da lista de correções e cria um dataframe com cada correção
correcoes_df = pd.DataFrame(correcoes, columns=['correcao_lev', 'correcao_ang', 'correcao_euc'])

# Cria uma coluna nova com as palavras incorretas
correcoes_df['palavra_incorreta'] = palavras_incorretas

# Extrai apenas a 5 primeiras palavras corrigidas de cada cálculo.
correcoes_df['correcao_lev'] = correcoes_df['correcao_lev'].apply(lambda x: ', '.join([palavra for palavra, _ in x[:5]]))
correcoes_df['correcao_ang'] = correcoes_df['correcao_ang'].apply(lambda x: ', '.join([palavra for palavra, _ in x[:5]]))
correcoes_df['correcao_euc'] = correcoes_df['correcao_euc'].apply(lambda x: ', '.join([palavra for palavra, _ in x[:5]]))

# Seleciona apenas as colunas desejadas
correcoes_df = correcoes_df[['palavra_incorreta', 'correcao_lev', 'correcao_ang', 'correcao_euc']]

display(correcoes_df)

,palavra_incorreta,correcao_lev,correcao_ang,correcao_euc
0,BERINJALA,"BERINJELA, BENGALA, SERINGAL, SERENATA, GERMINAL","LARANJEIRA, BALNEARIO, BERINJELA, ALEIJAR, ALI...","ALEIJAR, BERINJELA, BALNEARIO, ALIENAR, BALEEIRA"
1,ONOMATOPEYA,"ONOMATOPEIA, ONOMATOPEICO, FARMACOPEIA, ONOMAS...","ONOMATOPEIA, CAMPEONATO, POVOAMENTO, AMONTOAME...","ONOMATOPEIA, CAMPEONATO, POVOAMENTO, MAOMETANO..."
2,AXUL,"AZUL, ATOL, SUL, UL, ATUAL","XAUAL, LUXAR, XUA, LUA, LUX","XUA, LUA, LUXAR, LUX, XAUAL"
3,PARALIZAR,"PARALISAR, BANALIZAR, AVALIZAR, NASALIZAR, PAR...","PALIAR, POLARIZAR, PARALISAR, APRAZAR, ARRAIAL","ARRAIAL, PARALISAR, APRAZAR, POLARIZAR, PATRIA..."
4,ESPOJAR,"ESPOSAR, DESPOJAR, ESPOCAR, ESPORA, ESFOLAR","DESPOJAR, ESPORA, ARPEJO, ASPERO, SOPESAR","ESPORA, ARPEJO, DESPOJAR, ASPERO, SOBEJAR"
5,IMPECILHO,"EMPECILHO, IMBECIL, IMPERIO, IMPERIOSO, IMPRECISO","HIPOGLICEMIA, EMPECILHO, HEMOFILICO, POLICHINE...","EMPECILHO, EMPIRICO, POLEMICA, MILHEIRO, MILICO"
6,UVAAZ,"UVA, USADO, FUGAZ, VIA, UIVAR","VAZA, UVA, VAZAR, VAZIA, VAZAO","VAZA, SAUVA, UVA, VAZAR, VAZAO"
7,EXPECIME,"ESPECIME, ESPECIMEN, ESPECIE, EXPELIR, EXPERTISE","ESPECIME, PEIXE, ESPECIMEN, ESPECIE, MECE","ESPECIME, ESPECIMEN, ESPECIE, PEIXE, EPIDERME"
8,PROESTANTISMO,"PROTESTANTISMO, PROSELITISMO, ESTATISMO, DILET...","PROTESTANTISMO, MONASTERIO, MISANTROPO, OPTOME...","PROTESTANTISMO, SORTIMENTO, MISANTROPO, PRESTI..."
9,DESECAR,"DEFECAR, DESEJAR, DESERCAO, DESBOCAR, PESCAR","DESCARTE, DESERCAO, ASCENDER, DESAPARECER, DES...","DESCER, DESCARTE, DESERCAO, ASCENDER, ACEDER"
